In [1]:
import pandas as pd
import os
from scipy.sparse import csr_matrix
import numpy as np
from IPython.display import display_html
import warnings

import random

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns
%matplotlib inline

from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k
from lightfm import LightFM
from skopt import forest_minimize


/opt/anaconda3/lib/python3.8/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
%%time
anime = pd.read_csv('/Users/adrienbenoist/Desktop/Ironhack/GitHub/data-ft-par-labs/Projects/Anime dataset/anime.csv')
watching_status = pd.read_csv('/Users/adrienbenoist/Desktop/Ironhack/GitHub/data-ft-par-labs/Projects/Anime dataset/watching_status.csv')
liste = pd.read_csv('/Users/adrienbenoist/Desktop/Ironhack/GitHub/data-ft-par-labs/Projects/Anime dataset/animelist.csv')
rating=pd.read_csv('/Users/adrienbenoist/Desktop/Ironhack/GitHub/data-ft-par-labs/Projects/Anime dataset/rating_complete.csv')
synopsis = pd.read_csv('/Users/adrienbenoist/Desktop/Ironhack/GitHub/data-ft-par-labs/Projects/Anime dataset/anime_with_synopsis.csv')

CPU times: user 50.6 s, sys: 20.6 s, total: 1min 11s
Wall time: 1min 15s


In [24]:
display(rating.columns)
rating.shape

Index(['user_id', 'anime_id', 'rating'], dtype='object')

(57633278, 3)

In [25]:
display(liste.columns)
liste.shape

Index(['user_id', 'anime_id', 'rating', 'watching_status', 'watched_episodes'], dtype='object')

(109224747, 5)

In [26]:
display(anime.columns)
anime.shape

Index(['MAL_ID', 'Name', 'Score', 'Genders', 'English name', 'Japanese name',
       'Type', 'Episodes', 'Aired', 'Premiered', 'Producers', 'Licensors',
       'Studios', 'Source', 'Duration', 'Rating', 'Ranked', 'Popularity',
       'Members', 'Favorites', 'Watching', 'Completed', 'On-Hold', 'Dropped',
       'Plan to Watch', 'Score-10', 'Score-9', 'Score-8', 'Score-7', 'Score-6',
       'Score-5', 'Score-4', 'Score-3', 'Score-2', 'Score-1'],
      dtype='object')

(17562, 35)

In [31]:
display(synopsis)
synopsis.shape

,MAL_ID,Name,Score,Genders,sypnopsis
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ..."
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",ches are individuals with special powers like ...
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",It is the dark century and the people are suff...
...,...,...,...,...,...
16209,48481,Daomu Biji Zhi Qinling Shen Shu,Unknown,"Adventure, Mystery, Supernatural",No synopsis information has been added to this...
16210,48483,Mieruko-chan,Unknown,"Comedy, Horror, Supernatural",ko is a typical high school student whose life...
16211,48488,Higurashi no Naku Koro ni Sotsu,Unknown,"Mystery, Dementia, Horror, Psychological, Supe...",Sequel to Higurashi no Naku Koro ni Gou .
16212,48491,Yama no Susume: Next Summit,Unknown,"Adventure, Slice of Life, Comedy",New Yama no Susume anime.


(16214, 5)

In [4]:
liste_selected = liste[['user_id', 'anime_id', 'rating']]
liste_selected = liste_selected[liste_selected['user_id'].isin(random.sample(list(liste_selected['user_id'].unique()),k=5000))] 


In [5]:
user_anime_interaction = pd.pivot_table(liste_selected, index='user_id', columns='anime_id', values='rating')


In [6]:
user_id = list(user_anime_interaction.index)
user_dict = {}
counter = 0 
for i in user_id:
    user_dict[i] = counter
    counter += 1

In [7]:
user_anime_interaction = user_anime_interaction.fillna(0)

In [8]:
user_anime_interaction_csr = csr_matrix(user_anime_interaction.values)
user_anime_interaction_csr

<5000x16222 sparse matrix of type '<class 'numpy.float64'>'
	with 954746 stored elements in Compressed Sparse Row format>

In [9]:
model = LightFM(loss='warp',
                random_state=2016,
                learning_rate=0.90,
                no_components=150,
                user_alpha=0.000005)

model = model.fit(user_anime_interaction_csr,
                  epochs=100,
                  num_threads=16, verbose=False)

#model_lightfm = LightFM(loss='warp', learning_rate=0.01, k=10)
#model_lightfm.fit(train, item_features=anime_metadata_csr.T, epochs=50) 

In [10]:
item_dict ={}
df = anime[['MAL_ID', 'Name']].sort_values('MAL_ID').reset_index()

for i in range(df.shape[0]):
    item_dict[(df.loc[i,'MAL_ID'])] = df.loc[i,'Name']

In [11]:
anime.columns

Index(['MAL_ID', 'Name', 'Score', 'Genders', 'English name', 'Japanese name',
       'Type', 'Episodes', 'Aired', 'Premiered', 'Producers', 'Licensors',
       'Studios', 'Source', 'Duration', 'Rating', 'Ranked', 'Popularity',
       'Members', 'Favorites', 'Watching', 'Completed', 'On-Hold', 'Dropped',
       'Plan to Watch', 'Score-10', 'Score-9', 'Score-8', 'Score-7', 'Score-6',
       'Score-5', 'Score-4', 'Score-3', 'Score-2', 'Score-1'],
      dtype='object')

In [12]:
anime.Popularity

0           39
1          518
2          201
3         1467
4         4369
         ...  
17557    13116
17558    17562
17559    17558
17560    17565
17561    17563
Name: Popularity, Length: 17562, dtype: int64

In [13]:
anime.Score.replace('Unknown', '0', inplace=True)
anime.Score = pd.to_numeric(anime['Score'])
anime['Score'] = anime['Score'].apply(lambda x: round(x*2)/2)
anime_selected = anime[anime.MAL_ID.isin(liste_selected.anime_id.unique())]
bibtest = anime_selected[['Name', 'Score']]
bibtest= pd.get_dummies(bibtest, columns = ['Name', 'Score'])

In [14]:
anime_metadata_csr = csr_matrix(bibtest.values)
anime_metadata_csr

<16222x16235 sparse matrix of type '<class 'numpy.uint8'>'
	with 32444 stored elements in Compressed Sparse Row format>

In [15]:
user_anime_interaction.shape

(5000, 16222)

In [16]:
def sample_recommendation_user(model, liste, user_id, user_dict, 
                               item_dict,threshold = 0,nrec_items = 5, show = True):
    
    n_users, n_items = liste.shape
    user_x = user_dict[user_id]
    scores = pd.Series(model.predict(user_x,np.arange(n_items), item_features=anime_metadata_csr.T))
    scores.index = liste.columns
    scores = list(pd.Series(scores.sort_values(ascending=False).index))
    
    known_items = list(pd.Series(liste.loc[user_id,:] \
                                 [liste.loc[user_id,:] > threshold].index).sort_values(ascending=False))
    
    scores = [x for x in scores if x not in known_items]
    return_score_list = scores[0:nrec_items]
    known_items = list(pd.Series(known_items).apply(lambda x: item_dict[x]))
    scores = list(pd.Series(return_score_list).apply(lambda x: item_dict[x]))
    if show == True:
        print ("User: " + str(user_id))
        print("Known Likes:")
        counter = 1
        for i in known_items:
            print(str(counter) + '- ' + i)
            counter+=1

        print("\n Recommended Items:")
        counter = 1
        for i in scores:
            print(str(counter) + '- ' + i)
            counter+=1

In [17]:
user_dict.keys()

dict_keys([79, 220, 325, 367, 462, 485, 489, 529, 551, 663, 754, 793, 830, 854, 931, 1004, 1020, 1151, 1191, 1277, 1290, 1317, 1329, 1417, 1663, 1685, 1689, 1707, 1711, 1752, 1753, 1788, 1871, 2021, 2088, 2119, 2334, 2337, 2496, 2510, 2569, 2699, 2748, 2769, 2834, 2904, 3207, 3249, 3259, 3333, 3347, 3432, 3470, 3547, 3554, 3610, 3624, 3654, 3717, 3884, 3915, 3961, 3994, 4027, 4145, 4346, 4563, 4583, 4643, 4647, 4902, 4956, 4985, 5020, 5058, 5185, 5209, 5229, 5276, 5328, 5358, 5392, 5460, 5470, 5580, 5606, 5799, 5830, 5831, 5993, 6021, 6104, 6169, 6240, 6536, 6555, 6707, 6752, 6883, 6992, 7040, 7051, 7136, 7167, 7203, 7209, 7355, 7391, 7408, 7678, 7727, 7876, 7953, 7961, 7963, 7966, 7976, 8071, 8441, 8447, 8455, 8498, 8508, 8542, 8548, 8551, 8576, 8758, 8768, 9035, 9117, 9174, 9178, 9182, 9236, 9309, 9328, 9354, 9420, 9452, 9545, 9561, 9705, 9709, 9765, 9813, 9982, 10027, 10059, 10167, 10253, 10258, 10262, 10314, 10353, 10396, 10608, 10733, 10844, 10881, 10897, 10929, 11055, 11130, 1116

In [19]:
sample_recommendation_user(model, user_anime_interaction, 79, user_dict, item_dict)

User: 79
Known Likes:
1- Yuri!!! on Ice
2- Boku dake ga Inai Machi
3- Tokyo Ghoul √A
4- Shigatsu wa Kimi no Uso
5- Haikyuu!!

 Recommended Items:
1- Umi Monogatari: Marin no Kore Naani?
2- SK∞: Crazy Rock Jam
3- Koukaku Kidoutai Arise [.jp]
4- Churazima
5- Bouken Dankichi: Hyouryuu no Maki


In [ ]:
from lightfm import LightFM, cross_validation
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import auc_score, precision_at_k               #LIBRARIES (cpt Obvious)
train, test = cross_validation.random_train_test_split(user_anime_interaction_csr, test_percentage=0.25)           #TRAIN TEST (seems random then)
model_lightfm = LightFM(loss='warp', learning_rate=0.01, k=10)
model_lightfm.fit(train, item_features=anime_metadata_csr.T, epochs=50)                #MODELING (my PC can't)
train_precision = precision_at_k(model_lightfm, train, k=10, item_features=anime_metadata_csr.T).mean()
test_precision = precision_at_k(model_lightfm, test, k=10, item_features=anime_metadata_csr.T, train_interactions=train).mean()  #Precision_at_k score
train_auc = auc_score(model_lightfm, train, item_features=anime_metadata_csr.T).mean()
test_auc = auc_score(model_lightfm, test, item_features=anime_metadata_csr.T, train_interactions=train).mean()  #AUC score
print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))                        #Printing scores

In [ ]:
from lightfm import LightFM, cross_validation
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import auc_score, precision_at_k               #LIBRARIES (cpt Obvious)
train, test = cross_validation.random_train_test_split(anime_metadata_csr, test_percentage=0.25)           #TRAIN TEST (seems random then)
model_lightfm = LightFM(loss='warp', learning_rate=0.01, k=10)
model_lightfm.fit(train, item_features=anime_metadata_csr.T, epochs=50)                #MODELING (my PC can't)
train_precision = precision_at_k(model_lightfm, train, k=10, item_features=anime_metadata_csr.T).mean()
test_precision = precision_at_k(model_lightfm, test, k=10, item_features=anime_metadata_csr.T, train_interactions=train).mean()  #Precision_at_k score
train_auc = auc_score(model_lightfm, train, item_features=anime_metadata_csr.T).mean()
test_auc = auc_score(model_lightfm, test, item_features=anime_metadata_csr.T, train_interactions=train).mean()  #AUC score
print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))                        #Printing scores

In [ ]:
def hyperParameterTuning(X_train, y_train):
    LightFM_paramdict={
        'no_components':[50,150],
        'k':[5,10],
        'n':[5,15],
        'learning_rate':[0.1,0.5],
        'max_sampled':[5,10],
    }
    LFM_model = LightFM()
    gsearch = GridSearchCV(estimator = LFM_model,
                           param_grid = LightFM_paramdict,                        
                           scoring = 'precision_at_k',  
                           cv = 5,
                           n_jobs = -1,
                           verbose = 1)
    gsearch.fit(X_train, y_train)
    return gsearch.best_params_

In [ ]:
from sklearn.model_selection import GridSearchCV

print(gsearch.best_params_)
print(gsearch.best_score_)